# 使用gensim训练字向量

使用字向量的形式计算一个字前后是否要切分

In [2]:
import pymysql.cursors
from bs4 import BeautifulSoup
import re
import math

In [20]:
def get_connection():
    connection = pymysql.connect(host='localhost',
                                 user='root',
                                 password='qwe123',
                                 db='newlaw', 
                                 charset='utf8mb4')
    return connection

def pure_content(content):
    return re.sub("[\r\n 　]", "", content)

def remove_html_tags(content):
    return BeautifulSoup(pure_content(content), 'html.parser').get_text()

def corr(N, Nx, Ny, Nxy):
    pNx = math.log10(N/Nx)
    pNy = math.log10(N/Ny)
    pxy = Nxy / (Nx + Ny - Nxy)
    print("total = ", N, "\nNx = ", Nx, "\nNy = ", Ny,
          "\nNxy = ", Nxy, "\npNx = ", pNx, "\npNy = ", pNy,
          "\npxy = ", pxy)
    return pNx * pNy * pxy

In [17]:
def get_contents(conn):
    try:
        with conn.cursor() as cursor:
            sql = "select content " + \
            "from ex_news_contents where isEnglish = 'N'" + \
            " limit 20000"
            cursor.execute(sql)
            return cursor.fetchall()
    except BaseException as e:
        print(e)


In [15]:
contents = get_contents(get_connection())
sentences = []
doc_count = 0.0
chars = {}
words = {}
for content in contents:
    doc_count += 1
    pc = pure_content(remove_html_tags(content[0]))
    doc_chars = {}
    doc_words = {}
    for i in range(len(pc)-1):
        char1 = pc[i]
        char2 = pc[i+1]
        if char1 not in doc_chars:
            doc_chars[char1] = 1.0
        if char2 not in doc_chars:
            doc_chars[char2] = 1.0
        word = char1 + char2
        if word not in doc_words:
            doc_words[word] = 1.0

    for k, v in doc_chars.items():
        if k in chars:
            chars[k] += v
        else:
            chars[k] = 1.0

    for k, v in doc_words.items():
        if k in words:
            words[k] += v
        else:
            words[k] = 1.0

In [24]:
#print(chars, words)
#print(corr(doc_count, chars['法'], chars['规'], words['法规']))
#print(corr(doc_count, chars['规'], chars['定'], words['规定']))

print(corr(doc_count, chars['法'], chars['规'], words['法规']))
print(corr(doc_count, chars['规'], chars['职'], words['规职']))

total =  20000.0 
Nx =  13699.0 
Ny =  13003.0 
Nxy =  5044.0 
pNx =  0.1643411299917232 
pNy =  0.18698643311590377 
pxy =  0.23289315726290516
0.00715670464827076
total =  20000.0 
Nx =  13003.0 
Ny =  7936.0 
Nxy =  1.0 
pNx =  0.18698643311590377 
pNy =  0.4014283365178589 
pxy =  4.776005349125991e-05
3.5849485527330754e-06
